In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from pathlib import Path

import flexiznam as flz
from cottage_analysis.analysis import spheres
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, closed_loop_rsof, get_session_list
from v1_depth_analysis.v1_manuscript_2023 import common_utils as v1_common_utils

In [ ]:
VERSION = 5
SAVE_ROOT = Path(
    f"/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{VERSION}"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load example session
project = "hey2_3d-vision_foodres_20220101"
session_name = "PZAH6.4b_S20220524"
flexilims_session = flz.get_flexilims_session(project)

vs_df_example, trials_df_example = spheres.sync_all_recordings(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=project,
    filter_datasets={"anatomical_only": 3},
    recording_type="two_photon",
    protocol_base="SpheresPermTubeReward",
    photodiode_protocol=2,
    return_volumes=True,
)

neurons_ds_example = pipeline_utils.create_neurons_ds(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)
neurons_df_example = pd.read_pickle(neurons_ds_example.path_full)

In [ ]:
# Concatenate all sessions
project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)
results_all = v1_common_utils.concatenate_all_neurons_df(
    flexilims_session=flexilims_session,
    session_list=get_session_list.get_sessions(
        flexilims_session,
        closedloop_only=False,
        openloop_only=True,
    ),
    filename="neurons_df.pickle",
    cols=[
        "roi",
        "best_depth",
        "preferred_depth_closedloop",
        "preferred_depth_closedloop_crossval",
        "depth_tuning_test_rsq_closedloop",
        "depth_tuning_test_spearmanr_pval_closedloop",
        "depth_tuning_test_spearmanr_rval_closedloop",
        "preferred_RS_closedloop_g2d",
        "preferred_RS_closedloop_crossval_g2d",
        "preferred_OF_closedloop_g2d",
        "preferred_OF_closedloop_crossval_g2d",
        "rsof_rsq_closedloop_crossval_g2d",
        "rsof_rsq_closedloop_g2d",
        "preferred_RS_openloop_actual_g2d",
        "preferred_OF_openloop_actual_g2d",
        "rsof_rsq_openloop_actual_g2d",
        "rsof_popt_closedloop_g2d",
        "rsof_popt_openloop_actual_g2d",
    ],
    read_iscell=True,
    verbose=True,
)
results_all["preferred_RS_closedloop_g2d"] = (
    results_all["preferred_RS_closedloop_g2d"] * 100
)
results_all["preferred_depth_closedloop"] = (
    results_all["preferred_depth_closedloop"] * 100
)
results_all["preferred_OF_closedloop_g2d"] = np.degrees(
    results_all["preferred_OF_closedloop_g2d"]
)
results_all["preferred_RS_openloop_actual_g2d"] = (
    results_all["preferred_RS_openloop_actual_g2d"] * 100
)
results_all["preferred_OF_openloop_actual_g2d"] = np.degrees(
    results_all["preferred_OF_openloop_actual_g2d"]
)

In [ ]:
fig = plt.figure(figsize=(16 / 2.54, 14 / 2.54))
fontsize_dict = {"title": 5, "label": 7, "tick": 5, "legend": 5}
results_all["amplitude_closedloop"] = results_all["rsof_popt_closedloop_g2d"].apply(
    lambda x: np.exp(x[0])
)
results_all["amplitude_openloop"] = results_all["rsof_popt_openloop_actual_g2d"].apply(
    lambda x: np.exp(x[0])
)
# Filter neurons_df
neurons_df_sig = results_all[
    (results_all["iscell"] == 1)
    & (results_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    & (results_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (results_all["rsof_rsq_closedloop_g2d"] > 0.02)
]
scatter_props = {
    "s": 3,
    "alpha": 0.5,
    "plot_diagonal": True,
    "aspect_equal": True,
    "fontsize_dict": fontsize_dict,
}
closed_loop_rsof.plot_2d_hist(
    fig=fig,
    neurons_df=neurons_df_sig,
    xcol="preferred_RS_closedloop_g2d",
    ycol="preferred_RS_openloop_actual_g2d",
    xlabel="Preferred running speed \nin closed loop(cm/s)",
    ylabel="Preferred running speed \nin open loop (cm/s)",
    color="g",
    plot_x=0.68,
    plot_y=0.75,
    plot_width=0.18,
    plot_height=0.18,
    **scatter_props,
)

closed_loop_rsof.plot_2d_hist(
    fig=fig,
    neurons_df=neurons_df_sig,
    xcol="preferred_OF_closedloop_g2d",
    ycol="preferred_OF_openloop_actual_g2d",
    xlabel="Preferred optic flow speed \nin closed loop (degrees/s)",
    ylabel="Preferred optic flow speed \nin open loop (degrees/s)",
    color="b",
    plot_x=0.68,
    plot_y=0.45,
    plot_width=0.18,
    plot_height=0.18,
    **scatter_props,
)

# closed_loop_rsof.plot_scatter(
#     fig=fig,
#     neurons_df=neurons_df_sig,
#     xcol="amplitude_closedloop",
#     ycol="amplitude_openloop",
#     xlabel="Amplitude in closed loop",
#     ylabel="Amplitude in open loop",
#     c="r",
#     plot_x=0.68,
#     plot_y=0.15,
#     plot_width=0.18,
#     plot_height=0.18,
#     **scatter_props,
# )
# plt.xlim(plt.xlim()[0], 100)
# plt.ylim(plt.ylim()[0], 100)
# fig.add_axes([0.68, 0.15, 0.18, 0.18])
# plt.hist(
#     np.log10(
#         neurons_df_sig["amplitude_closedloop"] / neurons_df_sig["amplitude_openloop"]
#     ),
#     bins=np.linspace(-2, 2, 40),
#     color="k",
# )

# example open loop matrix, corner of RS 0 and OF 0
EXAMPLE_ROIS = [121, 440]  # 387,
# 312, 424, 477, 121, 446, 441, 324, 405
# 387, 357
# 293, 515, 440, 164
for iroi, roi in enumerate(EXAMPLE_ROIS):
    if iroi != (len(EXAMPLE_ROIS) - 1):
        xlabel = ""
    else:
        xlabel = "Running speed (cm/s)"
    if iroi != (len(EXAMPLE_ROIS) // 2):
        ylabel = ""
    else:
        ylabel = "Optic flow speed (degrees/s)"
    fig.add_axes([0.05, 0.75 - 0.3 * iroi, 0.17, 0.2])
    depth_selectivity.plot_depth_tuning_curve(
        neurons_df=neurons_df_example,
        trials_df=trials_df_example,
        roi=roi,
        rs_thr=None,
        plot_fit=False,
        linewidth=2,
        linecolor="k",
        fit_linecolor="r",
        closed_loop=1,
        fontsize_dict=fontsize_dict,
    )
    if iroi != len(EXAMPLE_ROIS) - 1:
        plt.xlabel("")

    vmin, vmax = closed_loop_rsof.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=1,
        vmin=None,
        vmax=None,
        xlabel="",
        ylabel=ylabel,
        plot_x=0.25,
        plot_y=0.75 - 0.3 * iroi,
        plot_width=0.2,
        plot_height=0.2,
        cbar_width=None,
        fontsize_dict=fontsize_dict,
    )

    closed_loop_rsof.plot_RS_OF_matrix(
        fig=fig,
        trials_df=trials_df_example,
        roi=roi,
        log_range={
            "rs_bin_log_min": 0,
            "rs_bin_log_max": 2.5,
            "rs_bin_num": 6,
            "of_bin_log_min": -1.5,
            "of_bin_log_max": 3.5,
            "of_bin_num": 11,
            "log_base": 10,
        },
        is_closed_loop=0,
        vmin=vmin,
        vmax=vmax,
        xlabel=xlabel,
        ylabel="",
        plot_x=0.4,
        plot_y=0.75 - 0.3 * iroi,
        plot_width=0.2,
        plot_height=0.2,
        cbar_width=0.005,
        fontsize_dict=fontsize_dict,
    )

plt.savefig(SAVE_ROOT / "fig4.pdf", dpi=300)
plt.savefig(SAVE_ROOT / "fig4.png", dpi=300)

In [ ]:
# use sign rank test to compare the amplitude in open loop and closed loop
from scipy.stats import wilcoxon

wilcoxon(neurons_df_sig["amplitude_closedloop"], neurons_df_sig["amplitude_openloop"])

In [ ]:
outliers = neurons_df_sig[neurons_df_sig["amplitude_closedloop"] > 20].copy()
# show columns roi and session
outliers[["roi", "session"]]

In [ ]:
# Minor:
# - add an illustration of closed loop & open loop
# - DONE: get rid of the speed plots?
# - DONE: separate out the two extra columns of the matrix
# - DONE: add dotted line for scatter plot
# - change spacing of matrix to 2